In [28]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [30]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore') #There are some suspicious ISBNs, we might loos them as they will not match with books on merging
ratings.rename(columns={'User-ID':'user'
                        , 'Book-Rating': 'rating'}, inplace=True)
ratings.shape
ratings.nunique()

(1149780, 3)

user      105283
ISBN      340553
rating        11
dtype: int64

In [33]:
# dtype_mapping = {
#     'Year-Of-Publication': float
# }
books = pd.read_csv('BX-Books.csv', sep = ';', encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')
books.drop(books[books['Unnamed: 8'].notna()].index, axis = 0, inplace=True) #Dropping the shifted rows. There is 19 of them, not worth the manual correction I think.
books.drop(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis = 1, inplace=True)
books.rename(columns={'Book-Title':'title'
                        , 'Book-Author': 'author'
                        , 'Year-Of-Publication': 'year'
                        , 'Publisher': 'publisher'}, inplace=True)
display(books.nunique())

C:\Users\ROB7441\AppData\Local\Temp\ipykernel_21132\1177692942.py:4: DtypeWarning: Columns (3,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep = ';', encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')


ISBN           271360
title          242133
author         102022
year              183
publisher       16806
Image-URL-S    271044
Image-URL-M    271044
Image-URL-L    271041
dtype: int64

In [4]:
users = pd.read_csv('BX-Users.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')
users.rename(columns={'User-ID':'user'
                        , 'Location': 'location'
                        , 'Age': 'age'}, inplace=True)
print(users.shape)
users.nunique()

(278858, 3)


user        278858
location     57293
age            165
dtype: int64

In [5]:
ratings

,user,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [52]:
#how many ratings each book have?
explicit_rating_counts = ratings[ratings['rating'] > 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().sort_values('rating').rename(columns={'rating':'explicit_ratings'})
implicit_rating_counts = ratings[ratings['rating'] == 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().sort_values('rating').rename(columns={'rating':'implicit_ratings'})
#implicit_rating_counts

if 'explicit_ratings' not in books.columns:
    books = books.merge(explicit_rating_counts, on = 'ISBN', how='left')
    books = books.merge(implicit_rating_counts, on = 'ISBN', how='left')

In [74]:
#Clearing the years of publication
books.drop(books[books['year'] == 'Gallimard'].index, axis = 0, inplace=True)
books.drop(books[books['year'] == 'DK Publishing Inc'].index, axis = 0, inplace=True)
books.loc[books.year == '0', 'year'] = np.NaN
books.loc[books.year == 0, 'year'] = np.NaN

In [82]:
books.year.unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', nan, '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', '1909', '2008', '1378', '1919', '1922',
       '1

In [88]:
books['year'] = books['year'].astype(int, errors='ignore')
books['year'].sum()

TypeError: can only concatenate str (not "int") to str

In [45]:
#Adding a time period when the book was released, instead of a year
#books['period1'] = (int(books['year']) // 10) * 10
#df['Second_Nearest_Tens'] = df['Nearest_Tens'] + ((df['Year'] % 10 >= 5) * 10)


In [53]:
books

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,NaN,NaN
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,NaN,NaN
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,NaN,NaN
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,NaN,NaN
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
271355,440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,NaN,NaN
271356,525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,NaN,NaN
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,NaN,1.0
271358,192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,NaN,NaN


In [62]:
#books[(books['author'].str.contains('Tolkien')).replace(np.NaN, False)]
books[(books['author'].str.contains('Robert Jordan')).replace(np.NaN, False)]

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings
2250,812513754,"Lord of Chaos (The Wheel of Time, Book 6)",Robert Jordan,1995,Tor Fantasy,http://images.amazon.com/images/P/0812513754.0...,http://images.amazon.com/images/P/0812513754.0...,http://images.amazon.com/images/P/0812513754.0...,NaN,NaN
2251,812550307,"The Fires of Heaven (The Wheel of Time, Book 5)",Robert Jordan,1994,Tor Fantasy,http://images.amazon.com/images/P/0812550307.0...,http://images.amazon.com/images/P/0812550307.0...,http://images.amazon.com/images/P/0812550307.0...,NaN,NaN
2252,812513738,"The Shadow Rising (The Wheel of Time, Book 4)",Robert Jordan,1993,Tor Fantasy,http://images.amazon.com/images/P/0812513738.0...,http://images.amazon.com/images/P/0812513738.0...,http://images.amazon.com/images/P/0812513738.0...,NaN,NaN
2253,312852487,"The Dragon Reborn (The Wheel of Time, Book 3)",Robert Jordan,1991,Tor Books,http://images.amazon.com/images/P/0312852487.0...,http://images.amazon.com/images/P/0312852487.0...,http://images.amazon.com/images/P/0312852487.0...,NaN,NaN
2254,765348446,"New Threads in the Pattern: The Great Hunt, Pa...",Robert Jordan,2003,Starscape,http://images.amazon.com/images/P/0765348446.0...,http://images.amazon.com/images/P/0765348446.0...,http://images.amazon.com/images/P/0765348446.0...,NaN,NaN
2255,765348438,"The Hunt Begins: The Great Hunt, Part 1 (The W...",Robert Jordan,2004,Starscape,http://images.amazon.com/images/P/0765348438.0...,http://images.amazon.com/images/P/0765348438.0...,http://images.amazon.com/images/P/0765348438.0...,NaN,NaN
2256,812511816,"The Eye of the World (The Wheel of Time, Book 1)",Robert Jordan,1990,Tor Fantasy,http://images.amazon.com/images/P/0812511816.0...,http://images.amazon.com/images/P/0812511816.0...,http://images.amazon.com/images/P/0812511816.0...,NaN,NaN
2824,765341840,"From The Two Rivers : The Eye of the World, Bo...",Robert Jordan,2002,Starscape,http://images.amazon.com/images/P/0765341840.0...,http://images.amazon.com/images/P/0765341840.0...,http://images.amazon.com/images/P/0765341840.0...,NaN,NaN
3002,812517725,"The Great Hunt (The Wheel of Time, Book 2)",Robert Jordan,1991,Tor Fantasy,http://images.amazon.com/images/P/0812517725.0...,http://images.amazon.com/images/P/0812517725.0...,http://images.amazon.com/images/P/0812517725.0...,NaN,NaN
3435,812513711,"The Dragon Reborn (The Wheel of Time, Book 3)",Robert Jordan,1992,Tor Fantasy,http://images.amazon.com/images/P/0812513711.0...,http://images.amazon.com/images/P/0812513711.0...,http://images.amazon.com/images/P/0812513711.0...,NaN,NaN


In [48]:
#books['info'] = books.

SyntaxError: invalid syntax (3376211048.py, line 1)